In [2]:
import pandas as pd

In [6]:
#Data Cleaning Step
df = pd.read_csv('glassdoor.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,0,0,VIE AUSTRALIA - Data Scientist IA,$93K - $134K (Glassdoor Est.),Organisation\nNaval Group compte parmi les act...,3.9,Naval Group\n3.9,Adelaide,5001 to 10000 Employees,1631,Company - Private,Aerospace & Defence,Aerospace & Defence,$2 to $5 billion (USD)
1,1,1,Specialist Data Scientist,-1,Specialist Data Scientist\nJoin us to help sol...,3.7,Rio Tinto\n3.7,Perth,10000+ Employees,1873,Company - Public,Mining & Metals,"Energy, Mining, Utilities",$10+ billion (USD)
2,2,2,Data Scientist,Employer Provided Salary:$85K - $150K,About the Role\nThe Data Scientist works under...,4.0,Telstra\n4.0,Melbourne,10000+ Employees,1901,Company - Public,Telecommunications Services,Telecommunications,$10+ billion (USD)
3,3,3,Data Scientist,Employer Provided Salary:$120K - $150K,Overview\nOur client is seeking experienced Da...,-1.0,Interpro People,Sydney,-1,-1,-1,-1,-1,-1
4,4,4,Data Science Intern,-1,YOUR OPPORTUNITY\nAre you interested in Data S...,3.6,InMoment\n3.6,Sydney,1001 to 5000 Employees,2002,Company - Private,Enterprise Software & Network Solutions,Information Technology,$25 to $50 million (USD)


In [7]:
#salary parsing
df['hourly'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
df['employer_provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'Employer Provided Salary:' in x.lower() else 0)

df = df[df['Salary Estimate']!= '-1']
salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0])
minus_kd = salary.apply(lambda x: x.replace('K', '').replace('$', ''))
min_hr = minus_kd.apply(lambda x: x.lower().replace('per hour', '').replace('employer provided salary:', '').replace(' ', ''))

df['cleaned_salary'] = min_hr
df['min_salary'] = min_hr.apply(lambda x: int(x.split('-')[0]))
df['max_salary'] = min_hr.apply(lambda x: int(x.split('-')[-1]))
df['avg_salary'] = (df.min_salary+df.max_salary)/2

In [8]:
df.hourly.value_counts()

0    714
1     62
Name: hourly, dtype: int64

In [4]:
#company name text only
df['Company Name'] = df.apply(lambda x: x['Company Name'] if x['Rating'] < 0 else x['Company Name'][:-4], axis = 1)

0    714
1     62
Name: hourly, dtype: int64

In [9]:
#state field. Changes NSW into Sydney
df['Location'] = df.apply(lambda x: x['Location'].replace('New South Wales', 'Sydney'), axis = 1)

In [10]:
#object types for Founded
print(df.dtypes)

Unnamed: 0.1           int64
Unnamed: 0             int64
Job Title             object
Salary Estimate       object
Job Description       object
Rating               float64
Company Name          object
Location              object
Size                  object
Founded               object
Type of ownership     object
Industry              object
Sector                object
Revenue               object
hourly                 int64
employer_provided      int64
cleaned_salary        object
min_salary             int64
max_salary             int64
avg_salary           float64
dtype: object


In [11]:
#To get age of the company we had to convert founded column into integers,
# however it contained some string so I had to remove them
# converted into a float, we'll see if this becomes an issue later

df['Founded_int'] = df.Founded.str.extract('(\d+)')
df = df.astype({"Founded_int": float})
print(df.dtypes)
df['age'] = df.Founded_int.apply(lambda x: -1 if x <2 else 2020 - x)

Unnamed: 0.1           int64
Unnamed: 0             int64
Job Title             object
Salary Estimate       object
Job Description       object
Rating               float64
Company Name          object
Location              object
Size                  object
Founded               object
Type of ownership     object
Industry              object
Sector                object
Revenue               object
hourly                 int64
employer_provided      int64
cleaned_salary        object
min_salary             int64
max_salary             int64
avg_salary           float64
Founded_int          float64
dtype: object


In [12]:
df_out = df.drop(['Unnamed: 0.1'], axis =1).drop(['Unnamed: 0'], axis =1)

In [13]:
df_out

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,hourly,employer_provided,cleaned_salary,min_salary,max_salary,avg_salary,Founded_int,age
0,VIE AUSTRALIA - Data Scientist IA,$93K - $134K (Glassdoor Est.),Organisation\nNaval Group compte parmi les act...,3.9,Naval Group\n3.9,Adelaide,5001 to 10000 Employees,1631,Company - Private,Aerospace & Defence,Aerospace & Defence,$2 to $5 billion (USD),0,0,93-134,93,134,113.5,1631.0,389.0
2,Data Scientist,Employer Provided Salary:$85K - $150K,About the Role\nThe Data Scientist works under...,4.0,Telstra\n4.0,Melbourne,10000+ Employees,1901,Company - Public,Telecommunications Services,Telecommunications,$10+ billion (USD),0,0,85-150,85,150,117.5,1901.0,119.0
3,Data Scientist,Employer Provided Salary:$120K - $150K,Overview\nOur client is seeking experienced Da...,-1.0,Interpro People,Sydney,-1,-1,-1,-1,-1,-1,0,0,120-150,120,150,135.0,1.0,-1.0
5,Data Scientist,$120K - $133K (Glassdoor Est.),Sportsbet's purpose is to bring excitement to ...,3.5,Sportsbet\n3.5,Melbourne,1001 to 5000 Employees,1993,Company - Private,Gambling & Betting,"Arts, Entertainment & Recreation",Unknown / Non-Applicable,0,0,120-133,120,133,126.5,1993.0,27.0
6,Senior Data Scientist,$127K - $170K (Glassdoor Est.),Join our innovative and collaborative Digital ...,4.1,Macquarie Group\n4.1,Sydney,10000+ Employees,1969,Company - Public,Investment & Asset Management,Finance,$5 to $10 billion (USD),0,0,127-170,127,170,148.5,1969.0,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,Analytics Lead,Employer Provided Salary:$144K - $166K,What is Drumline?\nDrumline Digital is an agen...,-1.0,Drumline Digital,Sydney,-1,-1,-1,-1,-1,-1,0,0,144-166,144,166,155.0,1.0,-1.0
994,Data Scientist,Employer Provided Salary:$120K - $150K,Overview\nOur client is seeking experienced Da...,-1.0,Interpro People,Sydney,-1,-1,-1,-1,-1,-1,0,0,120-150,120,150,135.0,1.0,-1.0
995,Data Analyst and Senior Data Analyst (multiple...,$100K - $148K (Glassdoor Est.),About the role\nJob Title: Data Analyst and Se...,4.1,ANZ Banking Group\n4.1,Melbourne,10000+ Employees,1835,Company - Public,Banking & Lending,Finance,Unknown / Non-Applicable,0,0,100-148,100,148,124.0,1835.0,185.0
998,Data Scientist,Employer Provided Salary:$85K - $150K,About the Role\nThe Data Scientist works under...,4.0,Telstra\n4.0,Melbourne,10000+ Employees,1901,Company - Public,Telecommunications Services,Telecommunications,$10+ billion (USD),0,0,85-150,85,150,117.5,1901.0,119.0


In [ ]:
df_out.to_csv('Salary_data_cleaned.csv', index = False)